In [2]:
import os
import sys
import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger('Logger')

In [3]:
from rucio.client import Client

In [4]:
import os
Rucio_cert = os.getenv('X509_USER_PROXY')
Rucio_key = os.getenv('X509_USER_PROXY')
Rucio_host = 'https://cms-rucio.cern.ch'
Rucio_account = 'belforte'
Rucio_authUrl = 'https://cms-rucio-auth.cern.ch'
Rucio_caPath = '/etc/grid-security/certificates/'

nativeClient = Client(
    rucio_host=Rucio_host, auth_host=Rucio_authUrl, ca_cert=Rucio_caPath,
    account=Rucio_account,
    creds={"client_cert": Rucio_cert, "client_key": Rucio_key},
    auth_type = 'x509'
)
ret=nativeClient.ping()
print(ret['version'])
ret = nativeClient.whoami()
print(ret)

1.23.11.post3
{u'status': u'ACTIVE', u'account': u'belforte', u'account_type': u'USER', u'created_at': u'2020-04-28T23:25:20', u'suspended_at': None, u'updated_at': u'2020-04-28T23:25:20', u'deleted_at': None, u'email': u'Stefano.Belforte@cern.ch'}


In [5]:
cli=nativeClient

In [6]:
site='T2_DE_DESY'
lfn1='/store/user/belforte/afile.root'

In [7]:
lfns=[lfn1]

In [8]:
# start with a dataset on tape
dataset = '/MuonEG/Run2016B-v1/RAW'

# can't use DBS Api in this Jupyter. Ask Rucio the list of blocks

blockNames=[]
response = cli.list_content('cms',dataset)
for item in response:
    if item['type'].upper() == 'DATASET':
        blockNames.append(item['name'])

logger.info('found %d blocks', len(blockNames))
import pprint
pprint.pprint(blockNames)


INFO:Logger:found 4 blocks
[u'/MuonEG/Run2016B-v1/RAW#86bc5e3e-1519-11e6-a3f4-001e67ac06a0',
 u'/MuonEG/Run2016B-v1/RAW#898359ee-1760-11e6-8d9e-001e67ac06a0',
 u'/MuonEG/Run2016B-v1/RAW#9da47a68-1682-11e6-8d9e-001e67ac06a0',
 u'/MuonEG/Run2016B-v1/RAW#cd4b1b62-13aa-11e6-a3f4-001e67ac06a0']


In [9]:
# now the locations
#dids = [{'scope':'cms','name':bname} for bname in blockNames]
#response = cli.list_dataset_replicas_bulk(dids)
scope = 'cms'
#result=[]
locationsMap = {}
for bName in blockNames:
    #print(bName)
    replicas = set()
    response = cli.list_dataset_replicas(scope, bName)
    for item in response:
        # same as complete='y' used for PhEDEx
        if item['state'].upper() == 'AVAILABLE':
            replicas.add(item['rse'])
    #result.append({'name': bName, 'replica': list(set(replicas))})
    locationsMap[bName] = list(replicas)
#pprint.pprint(result)
print("")
pprint.pprint(locationsMap)
#print("")
#lM = {}
#for r in result:
#    lM[r['name']] = r['replica']
#print(locationsMap==lM)


{u'/MuonEG/Run2016B-v1/RAW#86bc5e3e-1519-11e6-a3f4-001e67ac06a0': [u'T2_IT_Pisa',
                                                                   u'T1_FR_CCIN2P3_Tape',
                                                                   u'T0_CH_CERN_Tape'],
 u'/MuonEG/Run2016B-v1/RAW#898359ee-1760-11e6-8d9e-001e67ac06a0': [u'T2_IT_Pisa',
                                                                   u'T1_FR_CCIN2P3_Tape',
                                                                   u'T0_CH_CERN_Tape'],
 u'/MuonEG/Run2016B-v1/RAW#9da47a68-1682-11e6-8d9e-001e67ac06a0': [u'T2_IT_Pisa',
                                                                   u'T1_FR_CCIN2P3_Tape',
                                                                   u'T0_CH_CERN_Tape'],
 u'/MuonEG/Run2016B-v1/RAW#cd4b1b62-13aa-11e6-a3f4-001e67ac06a0': [u'T1_FR_CCIN2P3_Tape',
                                                                   u'T0_CH_CERN_Tape']}


In [13]:
myscope = 'user.belforte'
containerName = 'TapeRecall/pippo1/USER'
containerDid='user.belforte:/TapeRecall/pippo1/USER'

In [11]:
ret = cli.list_replication_rules({'did':containerDid})

AttributeError: 'Client' object has no attribute 'list_replication_rules'

In [14]:
ret = cli.list_did_rules(myscope, containerName)
                        

In [15]:
print(ret)

<generator object _load_json_data at 0x7fcf34030960>


In [16]:
resp=list(ret)

In [17]:
print(resp)

[{u'locks_ok_cnt': 0, u'source_replica_expression': None, u'locks_stuck_cnt': 0, u'purge_replicas': False, u'rse_expression': u'T3_IT_Trieste', u'updated_at': datetime.datetime(2021, 1, 7, 18, 5, 29), u'meta': None, u'child_rule_id': None, u'id': u'd2b715f526e14f91b0c299abb560d5d7', u'ignore_account_limit': False, u'error': None, u'weight': None, u'locks_replicating_cnt': 0, u'notification': u'NO', u'copies': 1, u'comments': u'Staged from tape for belforte', u'split_container': False, u'priority': 3, u'state': u'WAITING_APPROVAL', u'scope': u'user.belforte', u'subscription_id': None, u'stuck_at': None, u'ignore_availability': False, u'eol_at': None, u'expires_at': datetime.datetime(2021, 1, 21, 18, 5, 29), u'did_type': u'CONTAINER', u'account': u'belforte', u'locked': False, u'name': u'/TapeRecall/pippo1/USER', u'created_at': datetime.datetime(2021, 1, 7, 18, 5, 29), u'activity': u'Analysis Input', u'grouping': u'ALL'}]


In [18]:
print(resp[0]['id'])

d2b715f526e14f91b0c299abb560d5d7
